In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np
import io

In [2]:
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/cloud-apple-consumer/2019/20193' #subset of above
root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/cloud-apple-consumer*' # all directories

In [3]:
def extract_html_data(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    result = []
    
    for div in soup.find_all('div', {'class':'ac-gf-breadcrumbs-path'}):
        result.append(' '.join(div.text.split()))
    
    return result

In [4]:
def read_file(file):
    source = file.split('/')[4]
    archive = ZipFile(file, 'r')
    namelist = archive.namelist()
    ret = []
    for item in namelist:
        if 'html' in item and 'rss' not in item:
            html = archive.read(item)
            service = item.split('/')[1].split('.')[0]
            ts = item.split('/')[0]
            data = extract_html_data(html)
            data.insert(0, ts)
            data.insert(0, service)
            data.insert(0, source)
            ret.append(data)
    archive.close()
    return(ret)

In [5]:
files = glob.glob(root_dir + '/**/*.zip', recursive=True)
pool = mp.Pool(mp.cpu_count())
start_time = time.time()
results = pool.map(read_file, [file for file in files])
pool.close()
end_time = time.time()

In [6]:
print(end_time - start_time)

16.009714365005493


In [7]:
res = []
for outer in results:
    for inner in outer:
        res.append(inner)
print(len(res))

22306


In [8]:
column_names = ['source','service','timestamp','status']
df = pd.DataFrame(res, columns=column_names)
df.head(50)

,source,service,timestamp,status
0,cloud-apple-consumer,system-status,20200508T150001,System Status
1,cloud-apple-consumer,system-status,20200508T060001,System Status
2,cloud-apple-consumer,system-status,20200508T130001,System Status
3,cloud-apple-consumer,system-status,20200508T200001,System Status
4,cloud-apple-consumer,system-status,20200508T030001,System Status
5,cloud-apple-consumer,system-status,20200508T090001,System Status
6,cloud-apple-consumer,system-status,20200508T180001,System Status
7,cloud-apple-consumer,system-status,20200508T220001,System Status
8,cloud-apple-consumer,system-status,20200508T010001,System Status
9,cloud-apple-consumer,system-status,20200508T000001,System Status


In [17]:
df.loc[df['status'] != 'System Status'].count()

source       0
service      0
timestamp    0
status       0
dtype: int64

In [10]:
# df.to_csv(r'/home/shane/Documents/thesis/output/parsed/cloud-apple-consumer.csv', index=False)